In [1]:
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import zipfile
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_root = '../working/plates/'

In [3]:
os.getcwd()

In [4]:
print(os.listdir("../input/platesv2"))

In [5]:
with zipfile.ZipFile('../input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

In [6]:
img_folder = "/kaggle/working/plates"
train_folder = 'train'
test_folder = 'test'
os.listdir(os.path.join(img_folder, train_folder))

In [7]:
f = 100
# transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=0)
train_transforms_big = [transforms.Compose([
    transforms.CenterCrop(f),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    #transforms.RandomRotation(50),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(50),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.CenterCrop(f),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
transforms.Compose([
    transforms.RandomPerspective(distortion_scale=0.09, p=0.75, interpolation=3, fill=255),
    transforms.Resize((224, 224)),    
    transforms.ColorJitter(hue=(-0.5,0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
]

In [8]:
class CleanDirtyDataset(Dataset):
    def __init__(self, file_list, file_dir, mode, transform):
        self.file_list = file_list
        self.file_dir = file_dir
        self.mode = mode
        self.transform = transform
        
        if mode == 'train':
            if 'clean' in self.file_dir:
                self.label = 1 # clean label 
            else:
                self.label = 0 # dirty label
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.file_dir, self.file_list[idx]))
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[idx]
        

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
dirty_folder = os.path.join(img_folder, train_folder, 'dirty')
clean_folder = os.path.join(img_folder, train_folder, 'cleaned')
dirty = [f for f in os.listdir(dirty_folder) if '.jpg' in f]
clean = [f for f in os.listdir(clean_folder) if '.jpg' in f]

In [13]:
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader

clean_img_big = [CleanDirtyDataset(file_list=clean, 
                                   file_dir=clean_folder, 
                                   mode='train', 
                                   transform=transform) for transform in train_transforms_big]
dirty_img_big = [CleanDirtyDataset(file_list=dirty, 
                                   file_dir=dirty_folder, 
                                   mode='train', 
                                   transform=transform) for transform in train_transforms_big]

train_data_big = ConcatDataset(clean_img_big + dirty_img_big)

dataloader = DataLoader(train_data_big, batch_size=32, shuffle=True, num_workers=4)

In [14]:
import matplotlib.pyplot as plt

samples, labels = iter(dataloader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()
# in tensor, image is (batch, width, height), so you have to transpose it to (width, height, batch) in numpy to show it.
plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [21]:
from torchvision import models

device = "cuda"

model_res = models.resnet152(pretrained=True)

for param in model_res.parameters(): 
    param.requires_grad = False 
    
model_res.fc = torch.nn.Sequential(
    torch.nn.Linear(model_res.fc.in_features, 500),
    torch.nn.Linear(500, 2)
)
model_res = model_res.to(device)

loss = torch.nn.CrossEntropyLoss()

optimizer_sgd = torch.optim.SGD(model_res.parameters(), lr=0.01, momentum=0.95)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_sgd, step_size=7, gamma=0.1)

In [23]:
model = torchvision.models.densenet121(pretrained=True)

for param in model.parameters(): 
    param.requires_grad = False 
    
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(model.classifier.in_features, 500),
    torch.nn.Linear(500, 2)
)
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [24]:
epochs = 50
i = 1
check = len(dataloader) // 4
model.train()
total_loss = 0
loss_list = []
acc_list = []

for epoch in range(epochs):
    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        predict = model(imgs)
        loss_predict = loss(predict, labels)
        loss_predict.backward()
        optimizer.step()
        total_loss += loss_predict.item()
        scheduler.step()
        
        if i % check == 0:
            pred = torch.argmax(predict, dim=1)
            correct = pred.eq(labels)
            acc = torch.mean(correct.float())
            print(f'Epoch: {epoch}, loss: {total_loss}, acc: {acc}')
            loss_list.append(total_loss)
            acc_list.append(acc / check)
            total_loss = 0
        i += 1

In [25]:
plt.plot(loss_list)
plt.show()
plt.plot(acc_list)
plt.show()

In [26]:
test_transforms = {
                      'Вырезать в центре квадрат со стороной 140 ': transforms.Compose([
    transforms.CenterCrop(140),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                     'Вырезать в центре квадрат со стороной 135': transforms.Compose([
    transforms.CenterCrop(135),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]), 
                      'Вырезать в центре квадрат со стороной 130': transforms.Compose([
    transforms.CenterCrop(130),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 125': transforms.Compose([
    transforms.CenterCrop(125),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 120': transforms.Compose([
    transforms.CenterCrop(120),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 115': transforms.Compose([
    transforms.CenterCrop(115),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 110': transforms.Compose([
    transforms.CenterCrop(110),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 105': transforms.Compose([
    transforms.CenterCrop(105),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 100': transforms.Compose([
    transforms.CenterCrop(100),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                     'Вырезать в центре квадрат со стороной 95': transforms.Compose([
    transforms.CenterCrop(95),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                       'Вырезать в центре квадрат со стороной 90': transforms.Compose([
    transforms.CenterCrop(90),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                       'Вырезать в центре квадрат со стороной 85': transforms.Compose([
    transforms.CenterCrop(85),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                       'Вырезать в центре квадрат со стороной 80': transforms.Compose([
    transforms.CenterCrop(80),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
                      'Вырезать в центре квадрат со стороной 75': transforms.Compose([
    transforms.CenterCrop(75),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),                                         
                       'Вырезать в центре квадрат со стороной 70': transforms.Compose([
    transforms.CenterCrop(70),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),                                                           
                     }

In [27]:
import shutil
from tqdm import tqdm

In [28]:
batch_size = 10

In [29]:
img_folder

In [30]:
test_folder = os.path.join(img_folder, 'test')
test_imgs = [f for f in os.listdir(test_folder) if '.jpg' in f]

In [31]:
df = pd.DataFrame
# Для увеличения точности, необходимо предсказать результаты по разным вырезкам из изображения. 
# Вполне может случиться, что на меньшем изображении нет грязи, а на большем есть. 
# Тогда усреднение поможет исправить ситуацию.

for (_, transforms) in test_transforms.items():
    test_folder = os.path.join(img_folder, test_folder)
    test_imgs = [f for f in os.listdir(test_folder) if '.jpg' in f]
    test_data = CleanDirtyDataset(file_list=test_imgs, file_dir=test_folder, mode='test', transform=transforms)
    testloader = DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)

    model.eval()

    file_names = []
    res = []

    for x, fn in tqdm(testloader):
        with torch.no_grad():
            x = x.to(device)
            output = model(x)
    #         print(output)
#             pred = torch.argmax(output, dim=1)
    #         print(pred)
            file_names += fn
            res.append(torch.nn.functional.softmax(output, dim=1)[:,1].data.cpu().numpy())
#     test_folder = os.path.join(img_folder, test_folder)
#     test_imgs = [f for f in os.listdir(test_folder) if '.jpg' in f]
#     test_data = CleanDirtyDataset(file_list=test_imgs, file_dir=test_folder, mode='test', transform=transforms)
#     testloader = DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)
#     # Переводим модель в состояние eval 
#     model.eval()
# #     test_predictions = []  
# #     test_img_paths = [] 
#     # И предсказываем результаты
# #     for inputs, labels in tqdm(test_dataloader):
# #         inputs = inputs.to(device) 
# #         test_img_path.append(labels)
# #         with torch.set_grad_enabled(False):
# #             preds = model(inputs) 
# #         test_predictions.append(
# #             torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
# # #         test_img_paths.extend(paths)
# #     test_predictions = np.concatenate(test_predictions)
    
# #     # Записываем результаты
# #     submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
# # #     submission_df['id'] = submission_df['id']
# # #     submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
# #     submission_df.set_index('id', inplace=True)
#     ###
#     file_names = []
#     res = []

#     for x, fn in testloader:
#         with torch.no_grad():
#             x = x.to(device)
#             output = model(x)
#     #         print(output)
#             pred = torch.argmax(output, dim=1)
#     #         print(pred)
#             file_names += fn
#             res.append(torch.nn.functional.softmax(pred, dim=1)[:,1].data.cpu().numpy())

    img_id = [i.split('.')[0] for i in file_names]
    res = np.concatenate(res)
    final = pd.DataFrame({'id': img_id, 'label': res})

#     final['label'] = final['label'].map(lambda pred: 'dirty' if pred < 0.5 else 'cleaned')
    final.set_index('id', inplace=True)
    ###
    
    try : df = df.merge(final, how='inner', on='id') 
        
    # Для первой итерации
    except BaseException: 
        df = final 
df.head(8)

In [32]:
df.shape

In [33]:
len(test_transforms)

In [34]:
new_df = df.copy(deep=True)
new_df['mean'] = new_df.mean(axis=1)
new_df.drop(new_df.columns[:-1], axis='columns', inplace=True)
new_df['label'] = new_df['mean'].map(lambda pred: 'dirty' if pred < 0.50 else 'cleaned')
new_df.drop(new_df.columns[:-1], axis='columns', inplace=True)
new_df.head(13)

In [35]:
new_df.to_csv('submission.csv')

In [ ]:
new_df.set_index('id', inplace =True)

In [ ]:
test_transforms = transforms.Compose([
    transforms.CenterCrop(85),
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_folder = os.path.join(img_folder, test_folder)
test_imgs = [f for f in os.listdir(test_folder) if '.jpg' in f]
test_data = CleanDirtyDataset(file_list=test_imgs, file_dir=test_folder, mode='test', transform=test_transforms)
testloader = DataLoader(test_data, batch_size=4, shuffle=False, num_workers=4)

model.eval()

file_names = []
res = []

for x, fn in tqdm(testloader):
    with torch.no_grad():
        x = x.to(device)
        output = model(x)
#         print(output)
        pred = torch.argmax(output, dim=1)
#         print(pred)
        file_names += [n[:-4] for n in fn]
        res += [p.item() for p in pred]

# img_id = [i.split()[0] for i in file_names]
# final = pd.DataFrame({'id': img_id, 'label': res})

# final['label'] = final['label'].map(lambda pred: 'dirty' if pred < 0.5 else 'cleaned')
# final.set_index('id', inplace=True)

# final.to_csv('submission.csv')

